## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [3]:
print(tf.__version__)

2.0.0


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
#tf.enable_eager_execution()

### Collect Data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('/content/drive/My Drive/Great Learning/Residency 6/Day 2/Internal lab/prices.csv')

### Check all columns in the dataset

In [8]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

In [9]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [10]:
type(data)

pandas.core.frame.DataFrame

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [13]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [14]:
data.shape

(851264, 5)

### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
# split into input (X) and output (Y) variables
X = data.iloc[0:1000,0:4]
Y = data.iloc[0:1000,4]


In [16]:
X.head()

,open,close,low,high
0,123.430000,125.839996,122.309998,126.250000
1,125.239998,119.980003,119.940002,125.540001
2,116.379997,114.949997,114.930000,119.739998
3,115.480003,116.620003,113.500000,117.440002
4,117.010002,114.970001,114.089996,117.330002


In [17]:
Y.head()

0    2163600.0
1    2386400.0
2    2489500.0
3    2006300.0
4    1408600.0
Name: volume, dtype: float64

In [0]:
Y=Y/1000000

In [19]:
Y.head()

0    2.1636
1    2.3864
2    2.4895
3    2.0063
4    1.4086
Name: volume, dtype: float64

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

In [24]:
train_y.shape

(700,)

In [0]:
train_y=train_y.reshape(-1,1)

In [26]:
train_y.shape

(700, 1)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:

w1 = tf.Variable( tf.random.normal( [4,1] ) )
b1 = tf.Variable( tf.random.normal( [1] ) ) 

In [28]:
w1

<tf.Variable 'Variable:0' shape=(4, 1) dtype=float32, numpy=
array([[-0.18909468],
       [ 1.3657302 ],
       [-0.244599  ],
       [ 1.6396779 ]], dtype=float32)>

In [29]:
b1

<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([-1.0115081], dtype=float32)>

2.Define a function to calculate prediction

In [0]:

def prediction(X_train,W,B):
    out = tf.matmul( X_train,W ) + B
    return out

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b


## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [33]:
for i in range(100):
    i=i+5    
    w1, b1 = train(train_x, train_y, w1, b1)
    print('Current Training Loss on iteration', i, loss(train_y, prediction(train_x, w1, b1)))

('Current Training Loss on iteration', 5, <tf.Tensor: id=72, shape=(), dtype=float32, numpy=245.11786>)
('Current Training Loss on iteration', 6, <tf.Tensor: id=113, shape=(), dtype=float32, numpy=243.03902>)
('Current Training Loss on iteration', 7, <tf.Tensor: id=154, shape=(), dtype=float32, numpy=241.12317>)
('Current Training Loss on iteration', 8, <tf.Tensor: id=195, shape=(), dtype=float32, numpy=239.35753>)
('Current Training Loss on iteration', 9, <tf.Tensor: id=236, shape=(), dtype=float32, numpy=237.73024>)
('Current Training Loss on iteration', 10, <tf.Tensor: id=277, shape=(), dtype=float32, numpy=236.23062>)
('Current Training Loss on iteration', 11, <tf.Tensor: id=318, shape=(), dtype=float32, numpy=234.84851>)
('Current Training Loss on iteration', 12, <tf.Tensor: id=359, shape=(), dtype=float32, numpy=233.5748>)
('Current Training Loss on iteration', 13, <tf.Tensor: id=400, shape=(), dtype=float32, numpy=232.4009>)
('Current Training Loss on iteration', 14, <tf.Tensor:

### Get the shapes and values of W and b

In [34]:
w1

<tf.Tensor: id=4120, shape=(4, 1), dtype=float32, numpy=
array([[1.1111827],
       [2.7020202],
       [1.0491397],
       [2.9822273]], dtype=float32)>

In [35]:
b1

<tf.Tensor: id=4123, shape=(1,), dtype=float32, numpy=array([1.6253617], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [0]:
y_pred=prediction(test_x,w1,b1)

In [37]:
print('Predicted',y_pred[0])
print('Actual',test_y[0])

('Predicted', <tf.Tensor: id=4138, shape=(1,), dtype=float32, numpy=array([5.564335], dtype=float32)>)
('Actual', 0.9716)


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv('/content/drive/My Drive/Great Learning/Residency 6/Day 2/Internal lab/11_Iris.csv')

In [39]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


In [40]:
iris.shape

(150, 6)

In [41]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [42]:
iris.set_index('Id',inplace=True)
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [43]:
iris_encode=pd.get_dummies(iris, drop_first=False)
iris_encode.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
Id,,,,,,,
1,5.1,3.5,1.4,0.2,1,0,0
2,4.9,3.0,1.4,0.2,1,0,0
3,4.7,3.2,1.3,0.2,1,0,0
4,4.6,3.1,1.5,0.2,1,0,0
5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
x = iris_encode.iloc[:,:4]
y = iris_encode.iloc[:,4:7]

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)

In [45]:
print(x.head())
print(x.shape)

    SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
Id                                                          
1             5.1           3.5            1.4           0.2
2             4.9           3.0            1.4           0.2
3             4.7           3.2            1.3           0.2
4             4.6           3.1            1.5           0.2
5             5.0           3.6            1.4           0.2
(150, 4)


In [46]:
print(y.head())
print(y.shape)

    Species_Iris-setosa  Species_Iris-versicolor  Species_Iris-virginica
Id                                                                      
1                     1                        0                       0
2                     1                        0                       0
3                     1                        0                       0
4                     1                        0                       0
5                     1                        0                       0
(150, 3)


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# baseline model
def create_baseline():
    # create model
  model = Sequential()
  model.add(Dense(10, input_shape=(4,), activation='softmax', name='fc1'))
  #model.add(Dense(10, activation='softmax', name='fc2'))
  model.add(Dense(3, activation='softmax', name='output'))
  model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [0]:
  # baseline model
def hiddenlayer_2():
    # create model
  model = Sequential()
  model.add(Dense(10, input_shape=(4,), activation='softmax', name='fc1'))
  model.add(Dense(3, activation='softmax', name='fc2'))
  model.add(Dense(3, activation='softmax', name='output'))
  model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

### Model Training 

In [50]:
model = create_baseline()
print('Neural Network Model Summary: ')
print(model.summary())
# Train the model

model.fit(train_x, train_y, verbose=2, batch_size=5, epochs=200)


Neural Network Model Summary: 
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 10)                50        
_________________________________________________________________
output (Dense)               (None, 3)                 33        
Total params: 83
Trainable params: 83
Non-trainable params: 0
_________________________________________________________________
None
Train on 120 samples
Epoch 1/200
120/120 - 0s - loss: 1.2205 - accuracy: 0.3417
Epoch 2/200
120/120 - 0s - loss: 1.1927 - accuracy: 0.3417
Epoch 3/200
120/120 - 0s - loss: 1.1725 - accuracy: 0.3417
Epoch 4/200
120/120 - 0s - loss: 1.1564 - accuracy: 0.3417
Epoch 5/200
120/120 - 0s - loss: 1.1441 - accuracy: 0.3417
Epoch 6/200
120/120 - 0s - loss: 1.1351 - accuracy: 0.3417
Epoch 7/200
120/120 - 0s - loss: 1.1276 - accuracy: 0.3417
Epoch 8/200
120/120 - 0s - loss: 1.1223 - accuracy: 

### Model Prediction

In [51]:
results = model.evaluate(test_x, test_y)
print('Final test set loss: {:4f}'.format(results[0]))
print('Final test set accuracy: {:4f}'.format(results[1]))

30/30 [==============================] - 0s 617us/sample - loss: 1.1009 - accuracy: 0.3000
Final test set loss: 1.100853
Final test set accuracy: 0.300000


### Save the Model

In [52]:
model.save('R6_Iris_NN.ipynb')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: R6_Iris_NN.ipynb/assets
INFO:tensorflow:Assets written to: assets


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [53]:
model = hiddenlayer_2()
print('Neural Network Model Summary: ')
print(model.summary())

# Train the model
model.fit(train_x, train_y, verbose=2, batch_size=5, epochs=200)
results = model.evaluate(test_x, test_y)
print('Final test set loss: {:4f}'.format(results[0]))
print('Final test set accuracy: {:4f}'.format(results[1]))

Neural Network Model Summary: 
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 10)                50        
_________________________________________________________________
fc2 (Dense)                  (None, 3)                 33        
_________________________________________________________________
output (Dense)               (None, 3)                 12        
Total params: 95
Trainable params: 95
Non-trainable params: 0
_________________________________________________________________
None
Train on 120 samples
Epoch 1/200
120/120 - 0s - loss: 1.1386 - accuracy: 0.3417
Epoch 2/200
120/120 - 0s - loss: 1.1299 - accuracy: 0.3417
Epoch 3/200
120/120 - 0s - loss: 1.1230 - accuracy: 0.3417
Epoch 4/200
120/120 - 0s - loss: 1.1174 - accuracy: 0.3417
Epoch 5/200
120/120 - 0s - loss: 1.1133 - accuracy: 0.3417
Epoch 6/200
120/120 - 0s - loss: 1.1